Background 

Since the program launched on May 31, 2017, participants completed a Basics survey to share basic demographic information about themselves.  On October 22, 2019, six (6) American Community Survey Disability questions were added to the Basics Survey.  However, roughly 250k participants who enrolled between May 31, 2017 to October 21, 2019 were served the version of the Basics Survey without the 6 disability questions, leading to data missingness and under-estimation of disability in the AoU Cohort.  

To address this gap, these six disability surveys were served to participants as a separate Life Functioning Survey (LFS) for participants enrolled prior to October 2019.

Scope

Analyze to compare LFS answers and questions seen by participants with Questions and Answers seen by participants in the TheBasics survey.  Do participants see the same q/a pairs regardless of which survey they answered the question in?

# Set Up

In [ ]:
import pandas as pd
dataset = 'aou-res-curation-prod.2023q2r1_ptsc_rdr'
dataset2 = 'aou-res-curation-output-prod.C2022Q4R9'

The Basics: Disability 903517

PPI parent code of

    Disability: Blind 903574
    Disability: Deaf 903573
    Disability: Difficulty Concentrating 903575
    Disability: Dressing Bathing 903577
    Disability: Errands Alone 903578
    Disability: Walking Climbing 903576

# Get Data

In [ ]:
# ancestor_cid = "903517"

# basics_disability_df = pd.read_gbq(f"""
#         SELECT distinct person_id, observation_source_concept_id, observation_source_value
#                 , value_source_value, value_as_string, value_source_concept_id
#        -- FROM `{dataset}.concept` 
#         FROM  `{dataset}.concept_ancestor` ---on (concept_id=ancestor_concept_id)
#         join `{dataset}.observation` on (descendant_concept_id=observation_concept_id)
#         --join `{dataset}.observation_ext` using(observation_id)
#         WHERE observation_concept_id not in (40766240,43528428,1585389) --hcau vocab issues
#         AND ancestor_concept_id = {ancestor_cid}
#         --and concept_class_id='Module'
#         --and concept_name IN ('The Basics') 
#         --and src_id='PPI/PM'
#         and questionnaire_response_id is not null
#         """)
# basics_disability_df.head()

In [ ]:
ancestor_cid = "903517"

basics_disability_df = pd.read_gbq(f"""
        SELECT distinct person_id, observation_source_concept_id, observation_source_value
                , value_source_value, value_as_string, value_source_concept_id
        FROM  `{dataset2}.concept_ancestor` ---on (concept_id=ancestor_concept_id)
        join `{dataset2}.observation` on (descendant_concept_id=observation_concept_id)
        WHERE observation_concept_id not in (40766240,43528428,1585389) --hcau vocab issues
        AND ancestor_concept_id = {ancestor_cid}
        and questionnaire_response_id is not null
        """)
basics_disability_df.head()

In [ ]:
print(basics_disability_df.person_id.nunique())
basics_disability_df_gp = basics_disability_df.groupby(['observation_source_value','observation_source_concept_id','value_source_value', 'value_as_string', 'value_source_concept_id'], as_index = False).nunique()
#basics_disability_df_gp.columns = ['basics_'+c for c in basics_disability_df_gp.columns]

In [ ]:
lfs_survey_df = pd.read_gbq(f""" 
        SELECT DISTINCT o.person_id, observation_source_concept_id, observation_source_value
                , value_source_value, value_as_string, value_source_concept_id
        FROM `{dataset}.survey_conduct` sc
        LEFT JOIN `{dataset}.observation` o ON questionnaire_response_id = survey_conduct_id
        WHERE survey_source_value = 'lfs' 
        -- AND observation_id IS NULL
""")
lfs_survey_df

In [ ]:
print(lfs_survey_df.person_id.nunique())
lfs_survey_gp = lfs_survey_df.groupby(['observation_source_value','observation_source_concept_id','value_source_value', 'value_as_string', 'value_source_concept_id'], as_index = False).nunique()
#lfs_survey_gp.columns = ['lfs_'+c for c in lfs_survey_gp.columns ]

# Compare

In [ ]:
def transform(df,c = c):
    DF= df[df.observation_source_concept_id == c][cols].drop_duplicates()
    dd = pd.DataFrame()
    dd['question'] = DF['observation_source_concept_id'].astype('str')+'_'+DF['observation_source_value'].astype('str')
    dd['answer'] = DF['value_source_concept_id'].astype('str')+'_'+DF['value_source_value'].astype('str')+'_'+DF['value_as_string'].astype('str')
    dd = dd.to_dict('records')
    return DF, dd

In [ ]:
cols = ['observation_source_concept_id','observation_source_value','value_source_value'
        ,'value_source_concept_id','value_as_string']
for c in qc_sim_diff.basics_observation_source_concept_id.unique():
    print('\n\n'+str(c)+ ' '+str(basics_disability_df_gp[basics_disability_df_gp.observation_source_concept_id == c]['observation_source_value'].unique()))
    print('\n  Basics')
    df_basics, dd_basics= transform(basics_disability_df_gp)
    display(df_basics)
    print('  LFS')
    df_lfs, dd_lfs = transform(lfs_survey_gp)
    display(df_lfs)
    for col in df_basics.columns:
        print(f'Are the {col}s the same?:'); print(set(df_basics[col].values== df_lfs[col].values))
    print(f'\nBranching logic the same?:'+ str(dd_basics== dd_lfs).upper()); print('\n  In The Basics Survey'); display(dd_basics); print('\n  In LFS Survey');display(dd_lfs)

In [ ]:
def transform(df,c = c):
    DF= df[df.observation_source_concept_id == c][cols].drop_duplicates()
    dd = pd.DataFrame()
    dd['question_id_and_str'] = DF['observation_source_concept_id'].astype('str')+'_'+DF['observation_source_value'].astype('str')
    dd['answer_id_and_str'] = DF['value_source_concept_id'].astype('str')+'_'+DF['value_source_value'].astype('str')
    dd = dd.to_dict('records')
    return DF, dd

In [ ]:
cols = ['observation_source_concept_id','observation_source_value','value_source_value'
        ,'value_source_concept_id'#,'value_as_string'
       ]
for c in qc_sim_diff.basics_observation_source_concept_id.unique():
    print('\n\nFOR QUESTION: '+str(c)+ ' '+str(basics_disability_df_gp[basics_disability_df_gp.observation_source_concept_id == c]['observation_source_value'].unique()).upper())
    print('\n  In The Basics Survey')
    df_basics, dd_basics= transform(basics_disability_df_gp); display(df_basics)
    print('  In LFS Survey')
    df_lfs, dd_lfs = transform(lfs_survey_gp); display(df_lfs)
#     for col in df_basics.columns:
#         print(f'Are the {col}s the same?:'); print(set(df_basics[col].values== df_lfs[col].values))
    print(f'\n  Branching logic (question/answer pairs) the same?:'+ str(dd_basics== dd_lfs).upper())